We have the following HJB:

$$
0 = \sup_{(\pi,c)\in \mathcal{U}} \left\{ \partial_tV(w,t) + \partial_wV(w,t)\cdot(r + \pi_t(\mu-r)w - c_t) + \frac{1}{2}\partial_{ww}Vw^2\pi_t^2\sigma^2 + e^{-\rho t}U(c_t,t) \right\}, \quad w\geq0, w_0\geq 0, c_t \geq 0
$$

An closed-form solution can be found for specific $U(c,t)$ for example those with constant relative risk-aversion $U(c, t) = \frac{c^{1-\gamma}}{1-\gamma}$.
However, for more complicated utility functions a numerical solver is needed. 

One way fo doing this is using finite differences. Since the equation has a terminal condition our first instinct is to use an implicit scheme. While it is possible to create an explicit scheme by changing variables to $\tau = T-t$, we remember from numerical analysis that such a scheme may face stability issues.

The implicit scheme is as follows: 

### Step 1: Finite Differences
- discretise the spatial variable linearly: $w_j = 0 + j\Delta w$, with $j = 0,...,N$, $w_N = w_{max}$ for suitably large $w_{max}$.
- discretise the time variable linearly: $t_k = t_0 + k\Delta t$ with $k = 0,...,M$, $t_M = T$. For simplicity we take t_0 = 0 W.L.O.G.
- We use central difference for the 2nd order derivative and an upwinding derivative for the advection (1st order derivative) term to ensure stability.

Let $v_j^k$ be the value function evaluated at grid point $(w_j, t_k)$, $\alpha_j$ be the advection coefficient evaluated at $w_j$, $d_j$ the diffusion coefficient evaluated at $w_j$, and $\Theta^k$ be the constant term.
$$\begin{align*}
\alpha_j(\pi,c) &:= r + \pi(\mu -r)w_j - c \\
d_j(\pi,c) &:= \frac{1}{2}w_j^2\pi^2\sigma^2 \\
\Theta^k(\pi,c) &:= e^{-\rho t_{k}}U(c,t_{k})
\end{align*}$$

The standard backward Euler scheme for the HJB is thus:

$$\frac{v^{k}_j - v^{k-1}_j}{\Delta t} = - \sup_{(\pi,c)}\left[ D^{up}_w(v^{k-1}_j)\alpha_j(\pi,c) + D^+_wD^-_w(v^{k-1}_j)d_j(\pi,c)+ \Theta^{k-1}(\pi,c)\right]$$

Where:
- $D^{up}_w$ is the forward difference operator $D^+_w(v^k_j) = \frac{v^k_{j+1} - v^k_{j}}{\Delta w}$ if $\alpha_j <0$, and $D^-_w(v^k_j) = \frac{v^k_{j} - v^k_{j-1}}{\Delta w}$ if $\alpha_j >0$
- $D^+_wD^-_w$ is the second order central difference operator $D^+_wD^-_w(v^k)_{j} = \frac{v^k_{j+1} - 2v^k_{j} + v^k_{j-1}}{(\Delta w)^2}$ 

Taking the supremum accross the admissable set of controls is a non-linear optimisation problem because the diffusion term is quadratic in $\pi$. Therefore, we rely on policy iteration in order to find the optimal control at each time step.

### Step 2: Policy Iteration

Policy Iteration is a form of reinforcement learning used for iteratively choosing optimal control. At the PDE level policy iteration can be shown to converge on the optimal policy. If sufficiently careful with what discretisation you take, this convergence carries over to our finite difference scheme.

In practice we guess a policy at the required timestep, evaluate the value function given the policy, then find a new optimal policy based on that evaluation. We use an extra subscript to indicate the policy iteration. Let the inital guess policy be $(\pi_0, c_0)$. Let $v^{*k}_j$ denote the optimal value function determined from the previous time step.

$$\frac{v^{*k}_j - v^{k-1}_{j,0}}{\Delta t} = - \left[ D^{up}_w(v^{k-1}_{j,0})\alpha_j(\pi_0, c_0) + D^+_wD^-_w(v^{k-1}_{j,0})d_j(\pi_0, c_0)+ \Theta_{j}^{k-1}(\pi_0, c_0)\right]$$

This yields a linear system of equations we can solve for $\mathbf{v^{k-1}_0}$. We then find the optimal policy w.r.t the resulting value function. This is a non-linear optimisation problem.

$$(\pi_1,c_1) = \argmax_{(\pi,c)} \left[ D^{up}_w(v^{k-1}_{j,0})\alpha_j(\pi_0, c_0) + D^+_wD^-_w(v^{k-1}_{j,0})d_j(\pi_0, c_0)+ \Theta_{j}^{k-1}(\pi_0, c_0)\right]$$ 

We iterate this process until $||\mathbf{v^{k-1}_n}-\mathbf{v^{k-1}_{n-1}}|| < \epsilon $ and:

$$(\pi^*, c^*) := (\pi_{n+1}, c_{n+1}) = \argmax_{(\pi,c)} \left[ D^{up}_w(v^{k-1}_{j,n})\alpha_j(\pi_{n}, c_{n}) + D^+_wD^-_w(v^{k-1}_{j,n})d_j(\pi_{n}, c_{n})+ \Theta_{j}^{k-1}(\pi_{n}, c_{n})\right]$$ 



we can write the difference equation as 

$$
\begin{align*}
\frac{v^{*k}_{j} - v^{k-1}_{j,n}}{\Delta t} &= - \left[ \left(\frac{\alpha_j(\pi_{n-1}, c_{n-1})}{\Delta w} + \frac{d_j(\pi_{n-1}, c_{n-1})}{(\Delta w)^2}\right)v^{k-1}_{j+1} + \left(-\frac{\alpha_j(\pi_{n-1}, c_{n-1})}{\Delta w} - 2\frac{d_j(\pi_{n-1}, c_{n-1})}{(\Delta w)^2}\right)v^{k-1}_{j} + \frac{d_j(\pi_{n-1}, c_{n-1})}{(\Delta w)^2}v^{k-1}_{j-1} + \Theta^{k-1}(\pi_{n-1}, c_{n-1})\right], \quad \text{for } \alpha_j(\pi_{n-1}, c_{n-1}) <0\\

\frac{v^{*k}_{j} - v^{k-1}_{j,n}}{\Delta t} &= - \left[ \frac{d_j(\pi_{n-1}, c_{n-1})}{(\Delta w)^2}v^{k-1}_{j+1} + \left(\frac{\alpha_j(\pi_{n-1}, c_{n-1})}{\Delta w} - 2\frac{d_j(\pi_{n-1}, c_{n-1})}{(\Delta w)^2}\right)v^{k-1}_{j} + \left(\frac{d_j(\pi_{n-1}, c_{n-1})}{(\Delta w)^2}-\frac{\alpha_j(\pi_{n-1}, c_{n-1})}{\Delta w}\right)v^{k-1}_{j-1} + \Theta^{k-1}(\pi_{n-1}, c_{n-1})\right ], \quad \text{for } \alpha_j(\pi_{n-1}, c_{n-1}) >0
\end{align*}
$$


Reformulating in matrix notation we define a matrix $\mathbf{M}$ for $2 \leq j \leq N-1$

$$\begin{align*}
[M]_{j,j} &= 1 - \Delta t\left(\frac{|\alpha_j(\pi_{n-1},c_{n-1})|}{\Delta w} - 2\frac{d_j(\pi_{n-1},c_{n-1})}{(\Delta w)^2}\right) \\
[M]_{j,j+1} &= - \Delta t\left(1_{\alpha<0}\frac{\alpha_j(\pi_{n-1}, c_{n-1})}{\Delta w} + \frac{d_j(\pi_{n-1}, c_{n-1})}{(\Delta w)^2}\right) \\
[M]_{j,j-1} &= - \Delta t\left(1_{\alpha>0}\frac{d_j(\pi_{n-1}, c_{n-1})}{(\Delta w)^2} -\frac{\alpha_j(\pi_{n-1}, c_{n-1})}{\Delta w} \right)\\
[M]_{1,\cdot} &= [1, 0, 0,...,0]\\
[M]_{N,\cdot} &=[0,..., -3,4,-1]\\
\end{align*}
$$

The first three lines are the difference equations and the final lines encode the Dirichlet boundary conditions at one end and the Neumann boundary conditions at the other. 

We define the vectors $\mathbf{v}^{*k}$ and $\mathbf{v}^{k-1}_{n-1}$
$$
\begin{align*}
[\mathbf{v}^{*k}]_j &= v^{*k}_j, \quad \text{for } 2 \leq j \leq N-1\\
[\mathbf{v}^{*k}]_1 &= 0\\
[\mathbf{v}^{*k}]_N &= 0\\\\
[\mathbf{v}^{k-1}_{n-1}] &= v^{k-1}_{j,n-1}, \quad \text{for } 1 \leq j \leq N\\
\end{align*}
$$

Finally we define the vector 
$$
\begin{align*}
[\mathbf{\Theta}^{k-1}(\pi_{n-1},c_{n-1})]_j = \Theta(\pi_{j,n-1}, c_{j,n-1}), \quad \text{for } 1 \leq j \leq N-1\\
[\mathbf{\Theta}^{k-1}(\pi_{n-1},c_{n-1})]_N = 0 
\end{align*}$$

Where $(\pi_{j,n-1}, c_{j,n-1})$ represent the (n-1)th policy iteration at the point $w_j$. 

And the difference equation becomes 
$$ \mathbf{v}^{*k} + \Delta t\Theta^{k-1}(\pi_{n-1},c_{n-1})= M\mathbf{v}^{k-1}_{n-1} $$

Noting that because $C(0,t) = 0$, consumption cannot exceed wealth, the top line reads correctly as the Dirichlet boundary condition and in order to make the bottom line read correctly as the Neumann boundary condition we set the last component of the vector $[\mathbf{v}^{*k} + \Delta t\Theta^{k-1}(\pi_{n-1},c_{n-1})]_{N}$ to 0.

In [50]:
import numpy as np
from scipy.optimize import minimize
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

# Wealth grid
w_max = 1e8
N = 50
w = np.linspace(0.0, w_max, N)
dw = w[1] - w[0]

# Time grid
T = 100.0
t0 = 20.0
dt = 0.1
t = np.arange(t0, T + 1e-12, dt)   # include endpoint tolerance

# Constants
gamma = 0.5
r = 0.03
rho = 0.03
sigma = 0.2
mu = 0.05

# policy iteration constants
max_iter = 10
tol = 1e-6

# Bequest function: returns array of length N evaluated at grid w
def B(w_grid, T):
    # example: zero bequest
    return np.zeros_like(w_grid)

# Utility function (vectorized)
def U_func(c, t_local):
    # ensure no negative consumption
    c = np.maximum(c, 0.0)
    return np.exp(-rho * t_local) * (c ** (1.0 - gamma)) / (1.0 - gamma)

# alpha and d (vectorized)
def alpha_func(w_grid, pi, c):
    # r - (mu-r) * pi - c
    return r - (mu - r) * pi - c

def d_func(w_grid, pi, c):
    # 0.5 sigma^2 w^2 pi^2
    return 0.5 * (sigma ** 2) * (w_grid ** 2) * (pi ** 2)

# theta: local term inside the RHS / Hamiltonian (vectorized)
def theta_func(pi, c, t_local):
    return U_func(c, t_local)

# Build M matrix for a given policy: alpha and d are arrays of length N
def M_func(alpha_arr, d_arr):
    alpha_arr = np.asarray(alpha_arr)
    d_arr = np.asarray(d_arr)
    if alpha_arr.shape[0] != N or d_arr.shape[0] != N:
        raise ValueError("alpha and d must have length N")

    M_2 = 1.0 - dt * (np.abs(alpha_arr) / dw - 2.0 * d_arr / (dw ** 2))

    M_1 = np.zeros(N)
    M_3 = np.zeros(N)
    mask_pos = alpha_arr < 0.0
    mask_neg = alpha_arr >= 0.0

    # alpha >= 0
    M_1[mask_neg] = -dt * (alpha_arr[mask_neg] / dw + d_arr[mask_neg] / (dw ** 2))
    M_3[mask_neg] = -dt * (d_arr[mask_neg] / (dw ** 2))

    # alpha < 0
    M_1[mask_pos] = -dt * (d_arr[mask_pos] / (dw ** 2))
    M_3[mask_pos] = -dt * (d_arr[mask_pos] / (dw ** 2) - alpha_arr[mask_pos] / dw)

    # Build sparse tridiagonal (use LIL for row modification)
    A = diags([M_1[1:], M_2, M_3[:-1]], offsets=[-1, 0, 1], shape=(N, N), format="lil")

    # Dirichlet at j=0: u(0) = 0 (example)
    A[0, :] = 0.0
    A[0, 0] = 1.0

    # Neumann at j=N-1: use [-3, 4, -1] stencil on last three cols
    # ensure N >= 3
    if N >= 3:
        A[-1, :] = 0.0
        A[-1, -3] = -3.0
        A[-1, -2] = 4.0
        A[-1, -1] = -1.0
    else:
        raise ValueError("N must be at least 3 for the Neumann stencil")

    return A.tocsr()

# local Hamiltonian for pointwise minimization
# policy: length-2 array [pi, c]
# alpha_sign is the alpha from the *current policy evaluation* used to choose upwind direction
def local_H(policy, v_left, v_center, v_right, alpha_sign, w_j, t_j):
    pi_j, c_j = policy[0], policy[1]

    # choose upwind derivative according to alpha_sign (previous policy evaluation)
    if alpha_sign < 0:
        D_up = (v_right - v_center) / dw
    else:
        D_up = (v_center - v_left) / dw

    D_xx = (v_right - 2.0 * v_center + v_left) / (dw ** 2)

    # compute local coefficients for candidate policy
    a_j = alpha_func(w_j, pi_j, c_j)
    d_j = d_func(w_j, pi_j, c_j)
    th = theta_func(pi_j, c_j, t_j)

    # Hamiltonian H = D_up * a_j + D_xx * d_j + theta
    # minimize returns negative H (we want to maximize)
    return - (D_up * a_j + D_xx * d_j + th)

# Allocate v and set terminal condition
v = np.zeros((len(t), N))
v[-1, :] = B(w, T)   # last time row = bequest

optimal_policy = np.zeros((len(t), N, 2))

# Backward Euler loop over time steps: iterate idx from last index down to 1
for idx in range(len(t) - 1, 0, -1):
    v_next = v[idx, :].copy()       # value at t_k (known)
    v_curr = v_next.copy()          # initial guess for v_{k-1}
    curr_policy = np.zeros((N, 2))  # initial policy guess (pi,c)

    # choose non-zero initial policy to avoid solution flatlining 
    curr_policy[:, 0] = 0.5   # example: 50% invested
    curr_policy[:, 1] = w * 0.05  # consume 5% of wealth

    t_curr = t[idx]                 # time corresponding to v_next
    
    for it in range(max_iter):
        pi_n = curr_policy[:, 0]
        c_n = curr_policy[:, 1]

        alpha_n = alpha_func(w, pi_n, c_n)
        d_n = d_func(w, pi_n, c_n)
        M_n = M_func(alpha_n, d_n)
        U_n = U_func(c_n, t_curr)

        # solve linear system (backward Euler)
        v_new = spsolve(M_n, v_next + U_n)

        # convergence check in sup norm
        sup_norm = np.linalg.norm(v_curr - v_new, ord=np.inf)
        if sup_norm < tol:
            v[idx - 1, :] = v_new
            optimal_policy[idx - 1, :, :] = curr_policy
            break

        # otherwise update v_curr and improve policy pointwise
        v_curr = v_new.copy()

        # pointwise improvement: skip boundaries j=0 and j=N-1
        for j in range(1, N - 1):
            # alpha_sign from last evaluation is used to choose upwind stencil
            alpha_sign = alpha_n[j]

            # bounds: example simple bounds (adjust to your economic constraints)
            pi_bounds = (-5.0, 5.0)    # example: allow leverage
            c_bounds = (0.0, 1e6)      # consumption non-negative and capped
            bounds = [pi_bounds, c_bounds]

            # initial guess for this node
            x0 = curr_policy[j, :].copy()

            res = minimize(
                local_H,
                x0,
                args=(v_curr[j - 1], v_curr[j], v_curr[j + 1], alpha_sign, w[j], t_curr),
                method="L-BFGS-B",
                bounds=bounds,
                options={"maxiter": 100}
            )

            curr_policy[j, :] = res.x

    else:
        # if loop completes without break (no convergence)
        # store last iterate anyway
        v[idx - 1, :] = v_curr
        optimal_policy[idx - 1, :, :] = curr_policy
        print(f"did not converge. Last sup_norm {sup_norm}")

# end time loop
print(v)

did not converge. Last sup_norm 8.378978236578405e-06
did not converge. Last sup_norm 0.0004422312485985458
did not converge. Last sup_norm 0.01330732461065054
did not converge. Last sup_norm 0.7237877734005451
did not converge. Last sup_norm 17006.709558464587
did not converge. Last sup_norm 2076.872369289398
did not converge. Last sup_norm 98137.42520618439
did not converge. Last sup_norm 43538158.426716805
did not converge. Last sup_norm 118075867.6770935
did not converge. Last sup_norm 2113607548.4694824
did not converge. Last sup_norm 2603855700.1289062


/var/folders/c_/m3x98zjs2bb6cm1g0y7c6v2w0000gn/T/ipykernel_97254/1392433420.py:115: RuntimeWarning: overflow encountered in scalar multiply
  return - (D_up * a_j + D_xx * d_j + th)
/Users/sebastianodutola/.virtualenvs/merton/lib/python3.11/site-packages/scipy/optimize/_numdiff.py:686: RuntimeWarning: invalid value encountered in subtract
  df = [f_eval - f0 for f_eval in f_evals]
/Users/sebastianodutola/.virtualenvs/merton/lib/python3.11/site-packages/scipy/optimize/_numdiff.py:687: RuntimeWarning: overflow encountered in divide
  df_dx = [delf / delx for delf, delx in zip(df, dx)]
/var/folders/c_/m3x98zjs2bb6cm1g0y7c6v2w0000gn/T/ipykernel_97254/1392433420.py:102: RuntimeWarning: invalid value encountered in scalar subtract
  D_up = (v_right - v_center) / dw
/var/folders/c_/m3x98zjs2bb6cm1g0y7c6v2w0000gn/T/ipykernel_97254/1392433420.py:106: RuntimeWarning: invalid value encountered in scalar subtract
  D_xx = (v_right - 2.0 * v_center + v_left) / (dw ** 2)
/var/folders/c_/m3x98zjs2bb6

did not converge. Last sup_norm nan
did not converge. Last sup_norm nan
did not converge. Last sup_norm nan
did not converge. Last sup_norm nan
did not converge. Last sup_norm nan
did not converge. Last sup_norm nan
did not converge. Last sup_norm nan
did not converge. Last sup_norm nan
did not converge. Last sup_norm nan
did not converge. Last sup_norm nan
did not converge. Last sup_norm nan
did not converge. Last sup_norm nan
did not converge. Last sup_norm nan
did not converge. Last sup_norm nan


KeyboardInterrupt: 

In [2]:
import numpy as np
from scipy.optimize import minimize
from scipy.sparse import diags, lil_matrix, csr_matrix
from scipy.sparse.linalg import spsolve
#Wealth grid 
w_max = 1e8
N = 1000
w = np.linspace(0,w_max,N)
dw = w_max/N

#Time grid 
T = 100
t0 = 20
dt = 0.1
t = np.arange(t0, T, dt)

#Constants 
gamma = 0.5
r = 0.03
rho = 0.03
sigma = 0.2
mu = 0.05

#policy iteration constants
max_iter = 10
tol = 1e-6

#Bequest function 
def B(w,T):
    return np.array(N)

#Utility function
def U_func(c, t): 
    return np.exp(-rho*t)*c**(1-gamma)/(1-gamma)
 
def alpha(w, pi, c):
    "Returns advection coefficient for a fixed policy at the grid points."
    return r - (mu-r)*pi - c

def d(w,pi,c):
    "Returns diffusion coefficient for a fixed policy at the grid points."
    return 0.5*(sigma**2)*(w**2)*(pi**2)

def theta(pi, c, t): 
    "Returns constant coefficient for a fixed policy evaluated at the grid points."
    return np.exp(-t*rho)*U(c, t)

#In order to solve for the previous time step we need to solve a linear equation we construct the matrix M 
def M_func(alpha, d):

    M_2 = 1 - dt*(np.abs(alpha)/dw - 2*d/(dw**2))
    M_1 = np.zeros(N)
    M_3 = np.zeros(N)
    mask_neg = alpha < 0
    mask_pos = alpha > 0
    M_1[mask_neg] = -dt * (alpha[mask_neg]/dw + d[mask_neg]/(dw**2))
    M_3[mask_neg] = -dt * (d[mask_neg]/(dw**2))  
    
    M_1[mask_pos] = -dt * (d[mask_pos]/(dw**2))
    M_3[mask_pos] = -dt * (d[mask_pos]/(dw**2) - alpha[mask_pos]/dw)
    
    # build tridiagonal sparse matrix
    M = diags(
        diagonals=[M_1[1:], M_2, M_3[:-1]],
        offsets=[-1, 0, 1],
        format='lil'
    )

    #adjust for boundary conditions 
    M[0,:] = 0
    M[0,0] = 1
    M[-1,:] = 0
    M[-1, -3:] = [-3, 4, -1]
    return M.tocsr()

#local hamiltonian used in pointwise minimisation
def local_H(policy, v_left, v_centre, v_right, alpha_nj, w_j, t):
    if alpha_n[j] < 0:
        Dup = (v_right-v_centre)/dw
    else: 
        Dup = (v_centre-v_left)/dw
    D_xx = (v_right-2*v_centre+v_left)/dw**2
    alpha_j = alpha(w_j, policy[0], policy[1])
    d_j = d(w_j, policy[0],policy[1])
    theta_j = theta(policy[0], policy[1], t)
    return - (Dup*alpha_j +D_xx*d_j + theta_j)


v = np.empty([len(t),N])
#the row representing the last time step should be initialised with a bequest function in this case we set to zero
v[-1,N] = B(w,T)

#a 3d policy array where the last dimension is has values for pi(W,t),c(W,t)
optimal_policy = np.empty([len(t),N,2])

for i in range(len(t)):
    idx = len(t)-i-1
    vk = v[idx,:]
    v_curr = v[idx-1,:]

    #The policy iteration loop
    curr_policy = np.zeros([N, 2])
    t_curr = t[idx]
    for _ in range(max_iter):
        pi_n = curr_policy[:,0]
        c_n = curr_policy[:,1]

        #for a given policy we set up the linear system 
        alpha_n = alpha(w, pi_n, c_n)
        d_n = d(w, pi_n, c_n)
        M_n = M_func(alpha_n,d_n)
        U_n = U_func(c_n, t_curr)

        #we solve the system to get the iteration 
        v_n = spsolve(M_n,vk + U_n)

        #we check whether we have convergence 
        if np.linalg.norm(v_curr-v_n, ord=np.inf) < tol:
            v[idx-1,:] = v_n
            optimal_policy[idx-1, :, :] = curr_policy
            break
        else: 
            v_curr = v_n
            for j in range(1,N-1):
                res = minimize(
                    local_H,
                    curr_policy[j,:],
                    args=(v_curr[j-1],v_curr[j],v_curr[j+1], alpha_n[j],w[j], t[idx]),
                    method="BFGS"
                    )
                curr_policy[j,:] = res.x

        








